In [2]:
# Used for obtaining the training data
! python ./preprocessing/download_wordvecs.py --download_dir ./data
! python ./preprocessing/squad_preprocess.py --data_dir ./data


File glove.840B.300d.zip successfully loaded


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\temp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Will download SQuAD datasets to ./data
Will put preprocessed SQuAD datasets in ./data
File train-v1.1.json successfully loaded
Train data has 87599 examples total
Number of (context, question, answer) triples discarded due to char -> token mapping problems:  97
Number of (context, question, answer) triples discarded because character-based answer span is unaligned with tokenization:  1173
Number of (context, question, answer) triples discarded due character span alignment problems (usually Unicode problems):  23
Processed 86306 examples of total 87599

File dev-v1.1.json successfully loaded
Dev data has 10570 examples total
Number of (context, question, answer) triples discarded due to char -> token mapping problems:  0
Number of (context, question, answer) triples discarded because character-based answer span is unaligned with tokenization:  176
Number of (context, question, answer) triples discarded due character span alignment problems (usually Unicode problems):  0
Processed 10394 

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\temp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!

Preprocessing train: 100%|██████████| 442/442 [00:22<00:00, 20.06it/s]

Preprocessing dev: 100%|██████████| 48/48 [00:02<00:00, 16.10it/s]


From the paper:
"We use a max sequence length of 600 during training and a hidden state size of 200 for all recurrent
units, maxout layers, and linear layers. All LSTMs have randomly initialized parameters and an
initial state of zero. Sentinel vectors are randomly initialized and optimized during training. For
the dynamic decoder, we set the maximum number of iterations to 4 and use a maxout pool size of
16. We use dropout to regularize our network during training (Srivastava et al., 2014), and optimize
the model using ADAM (Kingma & Ba, 2014). All models are implemented and trained with
Chainer (Tokui et al., 2015)."

In [1]:
from model import DynamicCoattentionNW
import chainer as ch


max_seq_length = 600

hid_state_size = 200

dyn_dec_max_it = 4
maxout_pool_size = 16

dropout = 0.1

c:\Users\temp\anaconda3\envs\DLNLP-Project\lib\site-packages\chainer\_environment_check.py:72: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy) version 12.3.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  warnings.warn(msg.format(


In [3]:
from preprocessing.vocab import get_glove

glove_path = "./data/glove.840B.300d.txt"
glove_vector_size = 300
emb_mat, word2id, id2word = get_glove(glove_path, glove_vector_size)

vocab_size = len(word2id)

print(emb_mat.shape)

Loading GLoVE vectors from file: ./data/glove.840B.300d.txt


100%|██████████| 2196017/2196017 [02:03<00:00, 17782.60it/s]

(2196018, 300)


In [52]:
%load_ext autoreload

%autoreload 2

from data_batcher import get_batch_generator
batchsize = 2
# batchsize = 128

train_context_path = "./data/train.context"
train_qn_path = "./data/train.question"
train_ans_path = "./data/train.answer"
train_span_path = "./data/train.span"

batch_gen = get_batch_generator(word2id, train_context_path, train_qn_path, train_span_path, batchsize, max_seq_length, max_seq_length, discard_long=True)

#temp
from model import DocAndQuesEncoder, CoattentionEncoder, DynamicPointingDecoder, DynamicCoattentionNW
# enc = DocAndQuesEncoder(emb_mat, dropout, hid_state_size)
# enc2 = CoattentionEncoder(dropout, hid_state_size)
# dec = DynamicPointingDecoder(dropout, hid_state_size, maxout_pool_size, dyn_dec_max_it)
model = DynamicCoattentionNW(max_seq_length, hid_state_size, dyn_dec_max_it, maxout_pool_size, dropout, emb_mat)

for batch in batch_gen:
    s, e = model.forward(batch.context_ids, batch.qn_ids)
    # D, Q = enc.forward(batch.context_ids, batch.qn_ids)
    # U = enc2.forward(D, Q)
    # s, e = dec.forward(U)
    print(s)
    print(e)


    print(batch.ans_span)
    # print(batch.ans_tokens)   
    break

# train_iter = ch.iterators.SerialIterator(train, batchsize)
# test_iter = ch.iterators.SerialIterator(test, batchsize, False, False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Refilling batches...
Refilling batches took 0.04088926315307617 seconds
(2, 600, 600)
(2, 600, 400)
[526 143]
[162 443]
[[ 95 100]
 [ 63  64]]


In [ ]:
model = DynamicCoattentionNW(max_seq_length, hid_state_size, dyn_dec_max_it, maxout_pool_size, dropout, emb_mat)

In [ ]:
# Setup an optimizer
optimizer = ch.optimizers.Adam()
optimizer.setup(model)


In [ ]:
# Create the updater, using the optimizer
updater = ch.training.StandardUpdater(train_iter, optimizer, device=-1)

In [ ]:
# Set up a trainer
max_epoch = 10
trainer = ch.training.Trainer(updater, (max_epoch, 'epoch'), out='result')

In [ ]:
#save model
#see https://docs.chainer.org/en/stable/guides/serializers.html